# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='../Reacher.x86')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

# 1. Start with setup an environment.
The ddpg agent is improved from depg_agent of ddpg-bipedal (udacity) which is implemented ddpg agent with single agent.

please check that the versions of the environment are ready or not. We can check the version requirements in "../python/requirements.txt"

Unity environment is already prepared for deep reinforcement learning at "../Reacher.x86"

In [ ]:
from unityagents import UnityEnvironment
from collections import deque
from ddpg_agent import Agent
from ddpg_agents import Agents
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:

env = UnityEnvironment(file_name='../Reacher.x86')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

# parameters
# number of episodes
n_episodes = 300
# Time limit 
time_limit = 1000

# 2. Define (multi-)agents
The baseline of ddpg_agent.py is agent of ddpg-bipedal.
The agent have been modified to apply a multi-agent learning framework.

In [ ]:
# agents = Agent(state_size=state_size, action_size=action_size, random_seed=0,n_agent=num_agents)
agents = Agents(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=0)


# 3. Define ddpg learning
Now I implement a code for ddpg learning.

In [ ]:
def ddpg(n_episodes=1000,time = 100):
    # scores is the result of learning.
    scores =[]
    scores_window = deque(maxlen =100)
    score_changes_to_plot = []
    for i_episode in range(n_episodes):
        # reset the environment.
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        agents.reset()
        # Learn a multi-agent at the same time.
        score =np.zeros(num_agents)

        # learning is ended if it takes too long time. If then, try to change time limit or random_seed value.
        # multi-agent system is easily failed with various reason. (local_minima)
        # Thus, I use for loop with time limit, not a while loop
        while True:
            # set action.
            actions = agents.act(states)
            # update step.
            env_info = env.step(actions)[brain_name]
            # get next state from environment observer.
            next_states = env_info.vector_observations
            # get reward from environment.
            rewards = env_info.rewards
            score += rewards
            # get done flag.
            dones = env_info.local_done
            # update agents by step().
            agents.step(states,actions,rewards,next_states, dones)
            # update state.
            states = next_states
            if np.any(dones):
                # skip to next episode.
                break
        scores.append(np.mean(score))
        scores_window.append(np.mean(score))
        score_changes_to_plot.append(np.mean(scores_window))
        # print scores every 100*N th episode
        if i_episode % 10 == 0:
            print('\rEpisode: \t{} \tScore: \t{:.2f} \tAverage Score: \t{:.2f}'.format(i_episode, np.mean(score), np.mean(scores_window)), end="")
        # if the agent is able to receive an average reward(over "100 episodes", and average of over all 20 agents) of at least +35. 
        if np.mean(scores_window) >= 30.0 and i_episode > 100:
            torch.save(agents.actor_local.state_dict(), 'checkpoint_action.pth')
            torch.save(agents.critic_local.state_dict(), 'checkpoint_critic.pth')

    return scores, score_changes_to_plot

# 4. Run ddpg algorithm!

In [ ]:
scores, score_changes_to_plot = ddpg(n_episodes=n_episodes, time=time_limit)

# 5. Plot the result of ddpg.

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title("Result of Project 2 Continuous Control (robot arm)")
plt.plot(np.arange(len(scores)), scores, label = "ddpg agent")
plt.plot(np.arange(len(scores)), score_changes_to_plot, label = "average of scores")
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
env.close()